In [1]:
import pandas as pd

path = "/home/rom42pla/repos/eeg_deep_emotion_recognition/checkpoints/20220722_103737_within_subject_k_fold/logs.csv"
df = pd.read_csv(path)
df

,Unnamed: 0,acc_arousal_train,epoch,acc_arousal_val,acc_liking_train,acc_liking_val,acc_mean_train,acc_dominance_val,acc_dominance_train,loss_train,loss_val,acc_mean_val,acc_valence_val,acc_valence_train,fold,subject
0,0,0.523750,0.0,0.81625,0.605625,1.0,0.576875,0.88750,0.621875,2.701323,2.076227,0.852500,0.70625,0.556250,0,s01
1,1,0.764375,1.0,0.81625,0.911250,1.0,0.785000,0.88750,0.820000,2.200725,1.928581,0.852500,0.70625,0.644375,0,s01
2,2,0.516250,0.0,0.85250,0.610000,1.0,0.565781,0.87125,0.614375,2.703539,2.062110,0.860312,0.71750,0.522500,1,s01
3,3,0.736875,1.0,0.85250,0.915000,1.0,0.780156,0.87125,0.828125,2.217161,1.906345,0.860312,0.71750,0.640625,1,s01
4,4,0.522500,0.0,0.80625,0.593750,1.0,0.575156,0.86625,0.616250,2.696255,2.111627,0.837187,0.67625,0.568125,2,s01
5,5,0.750625,1.0,0.80625,0.911875,1.0,0.781875,0.86625,0.821250,2.198807,1.974917,0.837187,0.67625,0.643750,2,s01


In [5]:
df.groupby(by="subject").max().mean()

Unnamed: 0             5.000000
acc_arousal_train      0.764375
epoch                  1.000000
acc_arousal_val        0.852500
acc_liking_train       0.915000
acc_liking_val         1.000000
acc_mean_train         0.785000
acc_dominance_val      0.887500
acc_dominance_train    0.828125
loss_train             2.703539
loss_val               2.111627
acc_mean_val           0.860312
acc_valence_val        0.717500
acc_valence_train      0.644375
fold                   2.000000
dtype: float64

In [15]:
import torch
a = torch.rand(128, 23, 128)
b = torch.rand(128, 4, 128).mT
print(a.shape, b.shape)
r = torch.bmm(a, b)
print(r.shape)
z = torch.rand(128, 4, 128)
print(z.shape)
r = torch.bmm(z, r)
print(r.shape)

torch.Size([128, 23, 128]) torch.Size([128, 128, 4])
torch.Size([128, 23, 4])
torch.Size([128, 4, 128])


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [128, 128] but got: [128, 23].